de V2 à V3 travail sur les logs , les exceptions et produire une image .nii.gz au lieu d'un fichier .npy

# Modèle ARIMA en R sur tous les voxels de la matière grise 

In [4]:
import nibabel as nib
import pandas as pd
import numpy as np

import nibabel as nib

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [12]:
import nipype.interfaces.io as nio           # Data i/o
import nipype.interfaces.spm as spm          # spm
import nipype.interfaces.matlab as mlab      # how to run matlab
import nipype.interfaces.utility as util     # utility
import nipype.pipeline.engine as pe          # pypeline engine

from nipype.interfaces.utility import Function, IdentityInterface


## Workflow start

In [13]:
# creation of a subworflow to calculate the arima residus
arimaResidus = pe.Workflow(name='arimaResidus')

## logging management 

In [14]:
from nipype import config
cfg = dict(logging=dict(workflow_level = 'DEBUG'),
           execution={'stop_on_first_crash': False,
                       'hash_method': 'content'})
config.update_config(cfg)
arimaResidus.config['execution'] = {'stop_on_first_rerun': 'True',
                                   'hash_method': 'timestamp'}
import os
from nipype import config, logging
config.update_config({'logging': {'log_directory': os.getcwd(),
                      'log_to_file': True }})
logging.update_logging(config)

from nipype import logging
iflogger = logging.getLogger('interface')
message = "Start of arima "
iflogger.info(message)

INFO:interface:Start of arima 


In [15]:
sourcedir = '/scratch/user/hirsch/datadir4/data_results_py'


from nipype import SelectFiles, Node
templates = dict(gmMask=sourcedir+ "/structural/normalized_files/" + "wc1*.nii",
                 bpfile=sourcedir+ "/functionnal/bandpassedFile/" + "wcra*_merged_bp.nii.gz")

filesource = Node(SelectFiles(templates), "filesource")
filesource.inputs.subject_id = "subj1"
filesource.outputs.get()

{'bpfile': <undefined>, 'gmMask': <undefined>}

bpfile = '/scratch/user/hirsch/datadir4/data_results_py/wcrat0009_epi_s04_d0001_merged_bp.nii.gz'
bp =nib.load(bpfile)
bparray = np.asarray(bp.dataobj).copy()

In [16]:
def computeGm(gmMask):
    
    import nibabel as nib
    import numpy as np
    import os
    
    from nipype import logging
    
    # on regarde le grey matter
    i1=nib.load(gmMask)         
    i1array=np.asarray(i1.dataobj).copy() 
    i1array[(i1array )< 0.2] = 0
    # binary mask the resulting image
    i1array[i1array > 0] = 1
    gm_coord = np.transpose(np.nonzero(i1array))
    #print "nb of gm voxels" + str(len(gm_coord))
    iflogger = logging.getLogger('interface')
    iflogger.info("nb of gm voxels" + str(len(gm_coord)))
    
    out_file = os.getcwd() + '/' + 'gm_coord_file.npy'
    np.save(out_file, gm_coord)
    return out_file
    

In [17]:
# identify all the gm voxels
computeGmVoxels = Node(Function(input_names=['gmMask'],
                                output_names=['out_file'],
                                function=computeGm),
                                name='computeGmVoxels')

arimaResidus.connect(filesource, "gmMask", computeGmVoxels, "gmMask")

DEBUG:workflow:(arimaResidus.filesource, arimaResidus.computeGmVoxels): No edge data
DEBUG:workflow:(arimaResidus.filesource, arimaResidus.computeGmVoxels): new edge data: {'connect': [('gmMask', 'gmMask')]}


In [19]:
def computeArimaResidu(gmPointsFile, signalFile):
    
    import nibabel as nib
    import numpy as np
    import pandas as pd
    import os
    
    from nipype import logging
    iflogger = logging.getLogger('interface')
    
    import rpy2.robjects as ro
    from rpy2.robjects import pandas2ri
    pandas2ri.activate()

    ro.r('library(stats)')
   
    gmPoints = np.load(gmPointsFile)
    print gmPoints.size
    print gmPoints[0]
    
    bp =nib.load(signalFile) 
    bparray = np.asarray(bp.dataobj).copy()
    
    # p.full((3, 5), 7, dtype=int)
    result = np.zeros(bparray.shape,  dtype=np.float)
    
    nb_errors_arima=0
    
    for i in range(len(gmPoints)):
        
        #print gmPoints[i][0], gmPoints[i][1],gmPoints[i][2]
    
        # time serie associated to point i
        bp_ts = bparray[gmPoints[i][0], gmPoints[i][1],gmPoints[i][2], : ]
        #print bp_ts
    
        # covert numpy array to panda
        l = len(bp_ts)
        index = ['Row'+str(j) for j in range(1, l+1)]
        df = pd.DataFrame(bp_ts, index=index)
        #print df
        rdf = pandas2ri.py2ri(df)
        ro.globalenv['r_timeseries'] = rdf
        
        try:
            # model the time serie with ARIMA
            ro.r('fit <- arima(r_timeseries, order = c(1,1,1))')
            # get the residu
            residu = ro.r('fit$residuals')
            # result update
            result[gmPoints[i][0], gmPoints[i][1],gmPoints[i][2], : ] = residu
        except:
            nb_errors_arima += 1
            iflogger.info("i, and exception arima / nb errors arima :" + str(i) + " ,  " + str(nb_errors_arima))
            #print "exception arima"
    
    iflogger.info( "End iteration on i" )
    iflogger.info( "nb errors arima : " + str(nb_errors_arima))
      
    # create the resulting image
    residu_image = nib.Nifti1Image(result, bp.affine, bp.header)
    # save the residu array
    out_file = os.getcwd() + '/' + 'arima_residu.nii.gz'
    nib.save(out_file, residu_image)
    
    return out_file
    
    
    
    
    

In [20]:
# identify all the gm voxels
computeArimaResiduNode = Node(Function(input_names=['gmPointsFile', 'signalFile'],
                                output_names=['out_file'],
                                function=computeArimaResidu),
                                name='computeArimaResiduNode')

computeArimaResiduNode.inputs.ignore_exception = True

arimaResidus.connect(computeGmVoxels,  'out_file', computeArimaResiduNode, "gmPointsFile")
arimaResidus.connect(filesource,  'bpfile', computeArimaResiduNode, "signalFile")

DEBUG:workflow:(arimaResidus.computeGmVoxels, arimaResidus.computeArimaResiduNode): No edge data
DEBUG:workflow:(arimaResidus.computeGmVoxels, arimaResidus.computeArimaResiduNode): new edge data: {'connect': [('out_file', 'gmPointsFile')]}
DEBUG:workflow:(arimaResidus.filesource, arimaResidus.computeArimaResiduNode): No edge data
DEBUG:workflow:(arimaResidus.filesource, arimaResidus.computeArimaResiduNode): new edge data: {'connect': [('bpfile', 'signalFile')]}


In [21]:
# data sink
datasink = pe.Node(nio.DataSink(), name='datasink')
datasink.inputs.base_directory = '/scratch/user/hirsch/datadir4/data_results_py'

# for motion correction  plot files
arimaResidus.connect(computeGmVoxels,  'out_file', datasink, 'functionnal.arima.gmVoxels')
arimaResidus.connect(computeArimaResiduNode,  'out_file', datasink, 'functionnal.arima.arimaResidus')

DEBUG:workflow:(arimaResidus.computeGmVoxels, arimaResidus.datasink): No edge data
DEBUG:workflow:(arimaResidus.computeGmVoxels, arimaResidus.datasink): new edge data: {'connect': [('out_file', 'functionnal.arima.gmVoxels')]}
DEBUG:workflow:(arimaResidus.computeArimaResiduNode, arimaResidus.datasink): No edge data
DEBUG:workflow:(arimaResidus.computeArimaResiduNode, arimaResidus.datasink): new edge data: {'connect': [('out_file', 'functionnal.arima.arimaResidus')]}


In [22]:
arimaResidus.run()

DEBUG:workflow:Creating flat graph for workflow: arimaResidus
DEBUG:workflow:expanding workflow: arimaResidus
DEBUG:workflow:processing node: arimaResidus.filesource
DEBUG:workflow:processing node: arimaResidus.computeGmVoxels
DEBUG:workflow:processing node: arimaResidus.computeArimaResiduNode
DEBUG:workflow:processing node: arimaResidus.datasink
DEBUG:workflow:finished expanding workflow: arimaResidus
INFO:workflow:['check', 'execution', 'logging']
DEBUG:workflow:PE: expanding iterables
DEBUG:workflow:Detected iterable nodes []
DEBUG:workflow:PE: expanding iterables ... done
DEBUG:workflow:Performing depth first search
INFO:workflow:Running serially.
DEBUG:workflow:Setting node inputs
INFO:workflow:Executing node filesource in dir: /tmp/tmpjltcd3/arimaResidus/filesource
DEBUG:workflow:[]
DEBUG:workflow:/tmp/tmpjltcd3/arimaResidus/filesource/_0x4c5a623eb12cf0edf069fb7d0d1e25ca.json
DEBUG:workflow:('updatehash, overwrite, always_run, hash_exists', False, None, True, False)
DEBUG:workflo

444162
[10 42 32]


INFO:interface:i, and exception arima / nb errors arima :57466 ,  2046
INFO:interface:i, and exception arima / nb errors arima :57467 ,  2047
INFO:interface:i, and exception arima / nb errors arima :57507 ,  2048
INFO:interface:i, and exception arima / nb errors arima :57508 ,  2049
INFO:interface:i, and exception arima / nb errors arima :57509 ,  2050
INFO:interface:i, and exception arima / nb errors arima :57510 ,  2051
INFO:interface:i, and exception arima / nb errors arima :57511 ,  2052
INFO:interface:i, and exception arima / nb errors arima :57512 ,  2053
INFO:interface:i, and exception arima / nb errors arima :57513 ,  2054
INFO:interface:i, and exception arima / nb errors arima :57549 ,  2055
INFO:interface:i, and exception arima / nb errors arima :57550 ,  2056
INFO:interface:i, and exception arima / nb errors arima :57551 ,  2057
INFO:interface:i, and exception arima / nb errors arima :57552 ,  2058
INFO:interface:i, and exception arima / nb errors arima :57553 ,  2059
INFO:i

RuntimeError: Workflow did not execute cleanly. Check log for details

INFO:interface:i, and exception arima / nb errors arima :60299 ,  2174
INFO:interface:i, and exception arima / nb errors arima :60300 ,  2175
INFO:interface:i, and exception arima / nb errors arima :60301 ,  2176
INFO:interface:i, and exception arima / nb errors arima :60302 ,  2177
INFO:interface:i, and exception arima / nb errors arima :60354 ,  2178
INFO:interface:i, and exception arima / nb errors arima :60355 ,  2179
INFO:interface:i, and exception arima / nb errors arima :60356 ,  2180
INFO:interface:i, and exception arima / nb errors arima :60357 ,  2181
INFO:interface:i, and exception arima / nb errors arima :60358 ,  2182
INFO:interface:i, and exception arima / nb errors arima :60359 ,  2183
INFO:interface:i, and exception arima / nb errors arima :60407 ,  2184
INFO:interface:i, and exception arima / nb errors arima :60408 ,  2185
INFO:interface:i, and exception arima / nb errors arima :60409 ,  2186
INFO:interface:i, and exception arima / nb errors arima :60410 ,  2187
INFO:i

# Brouillons

In [ ]:
%pwd


In [ ]:
gm_file = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/arima/gmVoxels/gm_coord_file.npy'
bpfile = '/scratch/user/hirsch/datadir4/data_results_py/functionnal/bandpassedFile/wcrat0009_epi_s04_d0001_merged_bp.nii.gz'


In [ ]:
computeArimaResidu(gm_file, bpfile)

In [ ]:
gm_file = '/scratch/user/hirsch/datadir4/data_results_py/structural/normalized_files/wc1t0009_t1_s03.nii'
gm_pts =  computeGm(gm_file)
print gm_pts.shape
print gm_pts
    
   

In [ ]:
# bpfile = '/scratch/user/hirsch/datadir4/data_results_py/wcrat0009_epi_s04_d0001_merged_bp.nii.gz'
bp =nib.load(bpfile) 
bparray = np.asarray(bp.dataobj).copy()
ts_bp =  bparray[10, 42,  39, :]
plt.plot(ts_bp)

In [ ]:
ts_bp

In [ ]:
# covert numpy array to panda
l = len(ts_bp)
index = ['Row'+str(i) for i in range(1, l+1)]
df = pd.DataFrame(ts_bp, index=index)

In [ ]:
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
pandas2ri.activate()

ro.r('library(stats)')

rdf = pandas2ri.py2ri(df)
ro.globalenv['r_timeseries'] = rdf

In [ ]:
rdf

## Modélisation Arima

In [ ]:
ro.r('fit <- arima(r_timeseries)')
print(ro.r('summary(fit)'))

In [ ]:
ro.r('fit <- arima(r_timeseries, order = c(1,1,1))')
print(ro.r('summary(fit)'))


## plotting des residus ARIMA

In [ ]:
residu = ro.r('fit$residuals')

In [ ]:
rcParams['figure.figsize'] = 15, 6
plt.plot(residu)

## plotting des autocorrelations des residus ARIMA

In [ ]:
fig = plt.figure(figsize=(15,6))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(residu, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(residu, lags=40, ax=ax2)

## Comparaison des paramètres p, d, q 

In [ ]:
ro.r('AIC(fit)')

In [ ]:
ro.r('fit1 <- arima(r_timeseries, order = c(1,4,1))')

In [ ]:
ro.r('AIC(fit1)')

## qqplot

In [ ]:
import statsmodels.api as sm
fig = plt.figure(figsize=(15,6))
ax = fig.add_subplot(111)
fig = sm.qqplot(residu, line='q', ax=ax, fit=True)